In [2]:
import os
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.feature import local_binary_pattern
from copy import deepcopy
import random
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img

from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
#import pickle
from sklearn.metrics import confusion_matrix
from keras import backend as K
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from keras.metrics import Accuracy

from keras.optimizers import SGD

In [4]:
def imshow(image, label = ""):
    plt.figure(figsize = (7, 7), dpi = 50)
    plt.xlabel(label, fontsize= 30)
    plt.imshow(image, cmap = 'gray')

In [5]:
def Save_Model(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")

In [6]:
train_datagen = ImageDataGenerator(validation_split=0.15)
test_datagen = ImageDataGenerator(validation_split=0.15)

In [7]:
train_data = train_datagen.flow_from_directory(
        r'./TRUNK12/',
        color_mode="rgb",
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        subset = 'training')

test_data = test_datagen.flow_from_directory(
        r'./TRUNK12/',
        color_mode="rgb",
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        subset = 'validation')

Found 341 images belonging to 12 classes.
Found 52 images belonging to 12 classes.


In [6]:
"""
x=np.concatenate([test_data.next()[0] for i in range(test_data.__len__())])
y=np.concatenate([test_data.next()[1] for i in range(test_data.__len__())])
"""

'\nx=np.concatenate([test_data.next()[0] for i in range(test_data.__len__())])\ny=np.concatenate([test_data.next()[1] for i in range(test_data.__len__())])\n'

In [7]:
#np.shape(x)

In [8]:
num_classes = 12

In [9]:
tf.config.list_physical_devices('GPU')

[]

In [14]:
featureLayer1=[Conv2D(64, (3, 3), padding='same',input_shape=(100, 100, 3)),
               Activation('relu'),
               BatchNormalization(),
               Conv2D(64, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               MaxPooling2D(pool_size=(2, 2))]

featureLayer2=[Conv2D(128, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               Conv2D(128, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               MaxPooling2D(pool_size=(2, 2))]

featureLayer3=[Conv2D(256, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               Conv2D(256, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               Conv2D(256, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               MaxPooling2D(pool_size=(2, 2))]

featureLayer4=[Conv2D(512, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               Conv2D(512, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               Conv2D(512, (3, 3), padding='same'),
               Activation('relu'),
               BatchNormalization(),
               MaxPooling2D(pool_size=(2, 2))]


fullConnLayer=[Flatten(),
               Dense(1024),
               Activation('relu'),
               Dropout(0.5),
               Dense(1024),
               Activation('relu'),
               Dropout(0.5)]

classificationLayer=[Dense(num_classes),
                     Activation('softmax')]

model = Sequential(featureLayer1 + featureLayer2 + featureLayer3 + featureLayer4 + fullConnLayer + classificationLayer)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      1792      
                                                                 
 activation (Activation)     (None, 100, 100, 64)      0         
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      36928     
                                                                 
 activation_1 (Activation)   (None, 100, 100, 64)      0         
                                                                 
 batch_normalization_1 (Batc  (None, 100, 100, 64)     256       
 hNormalization)                                        

In [15]:
"""
opt = keras.optimizers.Adam(learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,)
"""
opt = SGD(learning_rate= 0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [16]:
epochs = 100
es = EarlyStopping(monitor='val_accuracy',
                    mode='max',
                    patience=99,
                    verbose=1,
                    restore_best_weights=True,
                    baseline= 1)

In [17]:
history = model.fit(train_data,
                    batch_size= 32,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_data,
                    callbacks=[es],
                    shuffle=False)


Epoch 1/100
11/11 [==============================] - 59s 5s/step - loss: 4.3131 - accuracy: 0.1173 - val_loss: 2.5287 - val_accuracy: 0.0962
Epoch 2/100
11/11 [==============================] - 55s 5s/step - loss: 3.2198 - accuracy: 0.1789 - val_loss: 2.5770 - val_accuracy: 0.0962
Epoch 3/100
11/11 [==============================] - 52s 5s/step - loss: 2.7428 - accuracy: 0.2375 - val_loss: 2.5202 - val_accuracy: 0.0385
Epoch 4/100
11/11 [==============================] - 58s 5s/step - loss: 2.3741 - accuracy: 0.2698 - val_loss: 2.4716 - val_accuracy: 0.1346
Epoch 5/100
11/11 [==============================] - 58s 5s/step - loss: 2.0458 - accuracy: 0.4018 - val_loss: 2.5035 - val_accuracy: 0.1154
Epoch 6/100
11/11 [==============================] - 64s 6s/step - loss: 1.9821 - accuracy: 0.4135 - val_loss: 2.4470 - val_accuracy: 0.1346
Epoch 7/100
11/11 [==============================] - 64s 6s/step - loss: 1.7033 - accuracy: 0.4751 - val_loss: 2.4552 - val_accuracy: 0.1154
Epoch 8/100
1

In [18]:
Save_Model(model)

Saved model to disk


In [19]:
model.evaluate(train_data, verbose=2)

11/11 - 32s - loss: 8.2097e-04 - accuracy: 1.0000 - 32s/epoch - 3s/step


[0.0008209681836888194, 1.0]

In [20]:
model.evaluate(test_data, verbose=2)

2/2 - 5s - loss: 0.4943 - accuracy: 0.8462 - 5s/epoch - 2s/step


[0.4942842721939087, 0.8461538553237915]

In [21]:
model.save('model_v1.h5')